# Reprise du code de SEBA


In [5]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LinearSegmentedColormap
import pyproj
import os


In [6]:
# Liste de tes 4 fichiers GeoTIFF
geotiff_files = [
    'data_input/data_processed/ghs_gaza_pop_added_full_extent.tif',
    'data_input/data_processed/ghs_gaza_strip.tif', 
    'data_input/data_processed/ghs_geneva.tif',
    'data_input/data_processed/ghs_schaffhausen.tif'
]

In [14]:
# Créer le dossier de sortie s'il n'existe pas
os.makedirs('data_output/visualisation_3d', exist_ok=True)

print("=== DIAGNOSTIC COMPLET DES GEOTIFF ===\n")

for i, file in enumerate(geotiff_files):
    print(f"Fichier {i+1}: {file}")
    print("-" * 50)
    
    try:
        with rasterio.open(file) as src:
            # Métadonnées de base
            print(f"Dimensions: {src.width} x {src.height} pixels")
            print(f"Bandes: {src.count}")
            print(f"CRS: {src.crs}")
            print(f"Bounds: {src.bounds}")
            
            # Lire les données
            data = src.read(1)
            print(f"Type de données: {data.dtype}")
            print(f"Forme du array: {data.shape}")
            
            # Statistiques brutes
            print(f"Valeurs uniques (échantillon): {np.unique(data.flatten())[:10]}")
            print(f"Min brut: {np.min(data)}")
            print(f"Max brut: {np.max(data)}")
            print(f"Moyenne brute: {np.mean(data):.2f}")
            
            # Compter les valeurs spéciales
            n_nan = np.sum(np.isnan(data))
            n_neg = np.sum(data < 0)
            n_zero = np.sum(data == 0)
            n_pos = np.sum(data > 0)
            
            print(f"NaN: {n_nan}")
            print(f"Négatifs: {n_neg}")
            print(f"Zéros: {n_zero}")
            print(f"Positifs: {n_pos}")
            
            # Nettoyer les données (supprimer les négatifs)
            data_clean = np.where(data < 0, np.nan, data)
            data_clean = np.where(data_clean == 0, np.nan, data_clean)  # Optionnel : supprimer aussi les zéros
            
            n_valid = np.sum(~np.isnan(data_clean))
            print(f"Pixels valides après nettoyage: {n_valid}")
            
            if n_valid > 0:
                print(f"Min nettoyé: {np.nanmin(data_clean):.2f}")
                print(f"Max nettoyé: {np.nanmax(data_clean):.2f}")
                print(f"Moyenne nettoyée: {np.nanmean(data_clean):.2f}")
                print(f"Médiane nettoyée: {np.nanmedian(data_clean):.2f}")
                
                # Percentiles pour comprendre la distribution
                p90 = np.nanpercentile(data_clean, 90)
                p95 = np.nanpercentile(data_clean, 95)
                p99 = np.nanpercentile(data_clean, 99)
                print(f"90e percentile: {p90:.2f}")
                print(f"95e percentile: {p95:.2f}")
                print(f"99e percentile: {p99:.2f}")
            else:
                print("AUCUNE DONNÉE VALIDE !")
            
    except Exception as e:
        print(f"ERREUR lors de la lecture: {e}")
    
    print("\n")

print("=== DIAGNOSTIC TERMINÉ ===")
print("Si tu vois des données valides, on peut passer à l'étape suivante !")

=== DIAGNOSTIC COMPLET DES GEOTIFF ===

Fichier 1: data_input/data_processed/ghs_gaza_pop_added_full_extent.tif
--------------------------------------------------
Dimensions: 419 x 450 pixels
Bandes: 1
CRS: EPSG:4326
Bounds: BoundingBox(left=34.21874988686396, bottom=31.219583410589177, right=34.56791655214766, top=31.594583409079952)
Type de données: float32
Forme du array: (450, 419)
Valeurs uniques (échantillon): [  0.      172.71988 172.72037 172.72041 172.72069 172.72086 172.72093
 172.72113 172.72118 172.7213 ]
Min brut: nan
Max brut: nan
Moyenne brute: nan
NaN: 137796
Négatifs: 0
Zéros: 41845
Positifs: 8909
Pixels valides après nettoyage: 8909
Min nettoyé: 172.72
Max nettoyé: 632.09
Moyenne nettoyée: 234.94
Médiane nettoyée: 187.21
90e percentile: 370.55
95e percentile: 401.24
99e percentile: 452.09


Fichier 2: data_input/data_processed/ghs_gaza_strip.tif
--------------------------------------------------
Dimensions: 419 x 450 pixels
Bandes: 1
CRS: EPSG:4326
Bounds: BoundingBox

## Version retravaillée avec claude

In [15]:

# ================================
# PARAMÈTRES À AJUSTER
# ================================
downsample_factor = 2  # 1=max détail, 2=équilibré, 3=rapide
height_multiplier = 2.0  # Multiplicateur pour la hauteur des barres (1.0=normal, 2.0=double hauteur)
create_small_multiple = True  # True pour créer l'aperçu 2x2 en plus des exports individuels

# ================================
# CONFIGURATION
# ================================

# Créer le dossier de sortie
os.makedirs('data_output/visualisation_3d', exist_ok=True)

# Create custom colormap with brighter colors
custom_colors_darkmode = [
    (0.0, '#575247'),     # Light beige (start at 0)
    (0.1, '#691A1F'),     # Medium green (middle)
    (1.0, '#E78D38')      # Dark green (end at 1)
]
custom_colors = [
    (0.0, '#DCD8C9'),     # Light beige (start at 0)
    (0.1, '#95D0CF'),     # Medium green (middle)
    (1.0, '#A757A3')      # Dark green (end at 1)
]

def calculate_area_km2(bounds, crs):
    """Calcule la superficie en km² en tenant compte du CRS"""
    if crs.to_string().startswith('EPSG:4326'):  # WGS84
        # Conversion approximative pour les degrés en km
        # À ces latitudes (~31-47°N), 1° longitude ≈ 85-75 km, 1° latitude ≈ 111 km
        center_lat = (bounds.bottom + bounds.top) / 2
        km_per_deg_lon = 111.32 * np.cos(np.radians(center_lat))
        km_per_deg_lat = 111.32
        
        width_km = (bounds.right - bounds.left) * km_per_deg_lon
        height_km = (bounds.top - bounds.bottom) * km_per_deg_lat
        area_km2 = width_km * height_km
    else:
        # CRS projeté, unités probablement en mètres
        width_m = bounds.right - bounds.left
        height_m = bounds.top - bounds.bottom
        area_km2 = (width_m * height_m) / 1_000_000
    
    return area_km2

# First pass: find global min and max values and pixel sizes
print("=== ANALYSE GLOBALE ===")
all_data = []
pixel_sizes = []
areas_km2 = []

for file in geotiff_files:
    with rasterio.open(file) as src:
        data = src.read(1)
        # Garder les zéros, supprimer seulement les négatifs
        data = np.where(data < 0, np.nan, data)
        all_data.append(data)
        transform = src.transform
        pixel_size = (abs(transform[0]), abs(transform[4]))
        pixel_sizes.append(pixel_size)
        
        # Calcul correct de la superficie
        area = calculate_area_km2(src.bounds, src.crs)
        areas_km2.append(area)
        
        print(f"{file.split('/')[-1]}: {area:.1f} km²")

vmin = min(np.nanmin(data) for data in all_data)
vmax = max(np.nanmax(data) for data in all_data)

print(f"Échelle globale: min={vmin:.2f}, max={vmax:.2f}")
print(f"Hauteur des barres: {height_multiplier}x normale")

# Find the largest dimensions to maintain scale
max_pixel_size = max(max(sizes) for sizes in pixel_sizes)
scale_factors = [(max_pixel_size/px[0], max_pixel_size/px[1]) for px in pixel_sizes]

print("Facteurs d'échelle calculés:")
for i, (file, scale) in enumerate(zip(geotiff_files, scale_factors)):
    print(f"  {file.split('/')[-1]}: {scale[0]:.3f} x {scale[1]:.3f}")

def create_3d_plot(ax, data, scale, area_km2, clean_name, mode='light', individual=True):
    """Fonction pour créer un plot 3D"""
    
    # Select appropriate colormap based on mode
    colors = custom_colors_darkmode if mode == 'dark' else custom_colors
    custom_cmap = LinearSegmentedColormap.from_list('custom', colors)
    
    if individual:
        # Make both figure and axis background transparent (pour exports individuels)
        ax.patch.set_alpha(0.0)
        # Set axis pane colors to transparent
        ax.xaxis.pane.set_facecolor((0.0, 0.0, 0.0, 0.0))
        ax.yaxis.pane.set_facecolor((0.0, 0.0, 0.0, 0.0))
        ax.zaxis.pane.set_facecolor((0.0, 0.0, 0.0, 0.0))
    
    # Downsample the data
    data_downsampled = data[::downsample_factor, ::downsample_factor]
    
    # Create coordinates for bars
    y, x = np.meshgrid(np.arange(data_downsampled.shape[1]), 
                      np.arange(data_downsampled.shape[0]))
    x = x.flatten() * downsample_factor * scale[0]
    y = y.flatten() * downsample_factor * scale[1]
    z = np.zeros_like(x)
    dx = dy = downsample_factor * max(scale) * 0.95
    dz = data_downsampled.flatten() * height_multiplier  # Appliquer le multiplicateur de hauteur
    
    # Remove only NaN values (keep zeros)
    mask = ~np.isnan(dz)
    x, y, z, dz = x[mask], y[mask], z[mask], dz[mask]
    
    if len(dz) > 0:
        # Create the bar plot
        bars = ax.bar3d(x, y, z, dx, dy, dz, 
                        zsort='max',
                        shade=True)

        # Get the colors for all faces
        norm = plt.Normalize(vmin=vmin, vmax=vmax * height_multiplier)
        face_colors = custom_cmap(norm(dz))
        
        # Modify colors for each face (6 faces per bar)
        facecolors = np.zeros((len(dz) * 6, 4))
        for i in range(len(dz)):
            color = face_colors[i]
            dark_color = color * [0.5, 0.5, 0.5, 1.0]  # Face du dessous plus sombre
            facecolors[i*6:(i+1)*6] = color
            facecolors[i*6 + 2] = dark_color  # Face du dessous

        # Apply the modified colors
        bars.set_facecolors(facecolors)

        # Set boundaries with proper scaling
        x_min, x_max = np.min(x), np.max(x) + dx
        y_min, y_max = np.min(y), np.max(y) + dy
        x_padding = (x_max - x_min) * 0.02
        y_padding = (y_max - y_min) * 0.02
        
        # Set axis limits
        ax.set_xlim(x_min - x_padding, x_max + x_padding)
        ax.set_ylim(y_min - y_padding, y_max + y_padding)
        ax.set_zlim(vmin, vmax * height_multiplier * 0.8)
    
    ax.set_axis_off()
    ax.view_init(elev=30, azim=0)
    
    # Add title
    title = f"{clean_name}\nSurface: {area_km2:.1f} km²"
    fontsize = 14 if individual else 10
    ax.set_title(title, pad=20, fontsize=fontsize)
    
    return custom_cmap

# Create individual plots
print("\n=== CRÉATION DES EXPORTS INDIVIDUELS ===")
for idx, (file, data, scale, area) in enumerate(zip(geotiff_files, all_data, scale_factors, areas_km2)):
    print(f"\nProcessing {file}")
    
    # Clean name
    clean_name = file.split('/')[-1].replace('ghs_', '').replace('.tif', '')
    if 'gaza_pop_added_full_extent' in clean_name:
        clean_name = 'Gaza (étendu)'
    elif 'gaza_strip' in clean_name:
        clean_name = 'Gaza (bande)'
    elif 'geneva' in clean_name:
        clean_name = 'Genève'
    elif 'schaffhausen' in clean_name:
        clean_name = 'Schaffhouse'
    
    # Create both light and dark mode versions
    for mode in ['light', 'dark']:
        print(f"Creating {mode} mode version...")
        
        # Create figure with transparent background
        fig = plt.figure(figsize=(12, 10), dpi=300)
        fig.patch.set_alpha(0.0)
        ax = fig.add_subplot(111, projection='3d')
        
        custom_cmap = create_3d_plot(ax, data, scale, area, clean_name, mode, individual=True)
        
        # Add colorbar
        cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
        sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax * height_multiplier))
        cbar = fig.colorbar(sm, cax=cbar_ax)
        cbar.set_label('Densité de population\n(hab./km²)', fontsize=12)
        
        # Adjust layout
        plt.subplots_adjust(left=0.02, right=0.9)
        
        # Save with appropriate suffix
        suffix = '_dm' if mode == 'dark' else ''
        safe_name = clean_name.lower().replace(' ', '_').replace('(', '').replace(')', '')
        output_filename = f"data_output/visualisation_3d/{safe_name}{suffix}.png"
        plt.savefig(output_filename, 
                    dpi=300, 
                    bbox_inches='tight',
                    pad_inches=0.1,
                    transparent=True,
                    facecolor='none')
        print(f"  Saved: {output_filename}")
        
        plt.close()

# Create small multiple overview
if create_small_multiple:
    print("\n=== CRÉATION DE L'APERÇU 2x2 ===")
    
    for mode in ['light', 'dark']:
        fig = plt.figure(figsize=(16, 12), dpi=300)
        if mode == 'dark':
            fig.patch.set_facecolor('#2E2E2E')
        else:
            fig.patch.set_facecolor('white')
        
        fig.suptitle(f'Comparaison des densités de population (hauteur {height_multiplier}x)', 
                    fontsize=16, y=0.95)
        
        for idx, (data, scale, area, file) in enumerate(zip(all_data, scale_factors, areas_km2, geotiff_files)):
            # Clean name
            clean_name = file.split('/')[-1].replace('ghs_', '').replace('.tif', '')
            if 'gaza_pop_added_full_extent' in clean_name:
                clean_name = 'Gaza (étendu)'
            elif 'gaza_strip' in clean_name:
                clean_name = 'Gaza (bande)'
            elif 'geneva' in clean_name:
                clean_name = 'Genève'
            elif 'schaffhausen' in clean_name:
                clean_name = 'Schaffhouse'
            
            ax = fig.add_subplot(2, 2, idx + 1, projection='3d')
            custom_cmap = create_3d_plot(ax, data, scale, area, clean_name, mode, individual=False)
        
        # Add single colorbar
        cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
        sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax * height_multiplier))
        cbar = fig.colorbar(sm, cax=cbar_ax)
        cbar.set_label('Densité de population (hab./km²)', fontsize=12)
        
        # Adjust layout
        plt.subplots_adjust(wspace=0.1, hspace=0.1, left=0.05, right=0.9, top=0.9, bottom=0.05)
        
        # Save overview
        suffix = '_dm' if mode == 'dark' else ''
        overview_filename = f"data_output/visualisation_3d/overview_2x2{suffix}.png"
        plt.savefig(overview_filename, dpi=300, bbox_inches='tight')
        print(f"Aperçu 2x2 sauvé: {overview_filename}")
        
        plt.close()

print(f"\n=== TERMINÉ ===")
print(f"Paramètres utilisés:")
print(f"  - Facteur de sous-échantillonnage: {downsample_factor}")
print(f"  - Multiplicateur de hauteur: {height_multiplier}x")
print(f"  - Aperçu 2x2 créé: {create_small_multiple}")
print(f"Toutes les visualisations sauvées dans: data_output/visualisation_3d/")

=== ANALYSE GLOBALE ===
ghs_gaza_pop_added_full_extent.tif: 1384.9 km²
ghs_gaza_strip.tif: 1384.9 km²
ghs_geneva.tif: 718.3 km²
ghs_schaffhausen.tif: 1006.7 km²
Échelle globale: min=0.00, max=632.09
Hauteur des barres: 2.0x normale
Facteurs d'échelle calculés:
  ghs_gaza_pop_added_full_extent.tif: 1.000 x 1.000
  ghs_gaza_strip.tif: 1.000 x 1.000
  ghs_geneva.tif: 1.000 x 1.000
  ghs_schaffhausen.tif: 1.000 x 1.000

=== CRÉATION DES EXPORTS INDIVIDUELS ===

Processing data_input/data_processed/ghs_gaza_pop_added_full_extent.tif
Creating light mode version...
  Saved: data_output/visualisation_3d/gaza_étendu.png
Creating dark mode version...
  Saved: data_output/visualisation_3d/gaza_étendu_dm.png

Processing data_input/data_processed/ghs_gaza_strip.tif
Creating light mode version...
  Saved: data_output/visualisation_3d/gaza_bande.png
Creating dark mode version...
  Saved: data_output/visualisation_3d/gaza_bande_dm.png

Processing data_input/data_processed/ghs_geneva.tif
Creating light

### Check des valeurs

In [12]:
import rasterio
import numpy as np
from pyproj import Transformer, CRS
import matplotlib.pyplot as plt

# Tes fichiers
geotiff_files = [
    'data_input/data_processed/ghs_gaza_pop_added_full_extent.tif',
    'data_input/data_processed/ghs_gaza_strip.tif', 
    'data_input/data_processed/ghs_geneva.tif',
    'data_input/data_processed/ghs_schaffhausen.tif'
]

print("=== DIAGNOSTIC COMPLET ===\n")

def deg_to_km(lat1, lon1, lat2, lon2):
    """Calcul précis de distance en km entre deux points"""
    from math import radians, cos, sin, asin, sqrt
    
    # Convertir en radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Formule de Haversine
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Rayon de la Terre en km
    return c * r

for i, file in enumerate(geotiff_files):
    print(f"{'='*60}")
    print(f"FICHIER {i+1}: {file.split('/')[-1]}")
    print(f"{'='*60}")
    
    with rasterio.open(file) as src:
        print(f"CRS: {src.crs}")
        print(f"Dimensions: {src.width} x {src.height} pixels")
        print(f"Transform: {src.transform}")
        print(f"Bounds: {src.bounds}")
        
        # Conversion des bounds
        left, bottom, right, top = src.bounds
        print(f"\nCoordonnées géographiques:")
        print(f"  Sud-Ouest: {bottom:.6f}°N, {left:.6f}°E")
        print(f"  Nord-Est:  {top:.6f}°N, {right:.6f}°E")
        
        # Calcul des dimensions réelles
        if src.crs.to_string().startswith('EPSG:4326'):
            # Largeur: distance entre coins sud
            width_km = deg_to_km(bottom, left, bottom, right)
            # Hauteur: distance entre coins ouest
            height_km = deg_to_km(bottom, left, top, left)
            
            print(f"\nDimensions réelles (méthode Haversine):")
            print(f"  Largeur: {width_km:.2f} km")
            print(f"  Hauteur: {height_km:.2f} km")
            print(f"  Surface: {width_km * height_km:.2f} km²")
            
            # Vérification avec résolution de pixel
            pixel_width_deg = abs(src.transform[0])
            pixel_height_deg = abs(src.transform[4])
            center_lat = (bottom + top) / 2
            
            # Conversion approximative au centre de l'image
            km_per_deg_lon = 111.32 * np.cos(np.radians(center_lat))
            km_per_deg_lat = 111.32
            
            pixel_width_km = pixel_width_deg * km_per_deg_lon
            pixel_height_km = pixel_height_deg * km_per_deg_lat
            
            print(f"\nRésolution des pixels:")
            print(f"  Largeur pixel: {pixel_width_deg:.8f}° = {pixel_width_km*1000:.1f} m")
            print(f"  Hauteur pixel: {pixel_height_deg:.8f}° = {pixel_height_km*1000:.1f} m")
            
        # Analyse des données
        data = src.read(1)
        print(f"\nAnalyse des données:")
        print(f"  Type: {data.dtype}")
        print(f"  Min brut: {np.nanmin(data):.2f}")
        print(f"  Max brut: {np.nanmax(data):.2f}")
        
        # Nettoyer les données
        data_clean = np.where(data < 0, np.nan, data)
        valid_pixels = np.sum(~np.isnan(data_clean))
        total_pixels = data.size
        
        print(f"  Pixels totaux: {total_pixels:,}")
        print(f"  Pixels valides: {valid_pixels:,} ({valid_pixels/total_pixels*100:.1f}%)")
        
        if valid_pixels > 0:
            print(f"  Min valide: {np.nanmin(data_clean):.2f}")
            print(f"  Max valide: {np.nanmax(data_clean):.2f}")
            print(f"  Moyenne valide: {np.nanmean(data_clean):.2f}")
            
            # Surface couverte par les données valides
            if src.crs.to_string().startswith('EPSG:4326'):
                pixel_area_km2 = pixel_width_km * pixel_height_km
                covered_area_km2 = valid_pixels * pixel_area_km2
                print(f"  Surface couverte par les données: {covered_area_km2:.2f} km²")
        
        print()

print("\n=== COMPARAISON DES ÉCHELLES ===")
print("Pour vérifier si les territoires sont comparables:")

# Extraire les informations de chaque fichier
territories = []
for file in geotiff_files:
    with rasterio.open(file) as src:
        left, bottom, right, top = src.bounds
        width_km = deg_to_km(bottom, left, bottom, right)
        height_km = deg_to_km(bottom, left, top, left)
        area_km2 = width_km * height_km
        
        name = file.split('/')[-1].replace('ghs_', '').replace('.tif', '')
        territories.append({
            'name': name,
            'width_km': width_km,
            'height_km': height_km,
            'area_km2': area_km2,
            'center_lat': (bottom + top) / 2,
            'center_lon': (left + right) / 2
        })

# Trier par superficie
territories.sort(key=lambda x: x['area_km2'])

print("\nTerritoires triés par superficie:")
for t in territories:
    print(f"{t['name']:30} {t['area_km2']:8.1f} km² ({t['width_km']:5.1f} x {t['height_km']:5.1f} km)")
    print(f"{'':30} Centre: {t['center_lat']:.3f}°N, {t['center_lon']:.3f}°E")

print(f"\nRatio max/min superficie: {territories[-1]['area_km2'] / territories[0]['area_km2']:.1f}x")

print("\n=== RECOMMANDATIONS ===")
if territories[-1]['area_km2'] / territories[0]['area_km2'] > 10:
    print("⚠️  ÉCHELLES TRÈS DIFFÉRENTES - La comparaison sera difficile")
else:
    print("✅ Échelles raisonnablement comparables")

# Vérifier les régions géographiques
gaza_files = [t for t in territories if 'gaza' in t['name'].lower()]
swiss_files = [t for t in territories if any(x in t['name'].lower() for x in ['geneva', 'schaffhausen'])]

if gaza_files and swiss_files:
    print("📍 Régions détectées: Moyen-Orient (Gaza) + Europe (Suisse)")
    print("   → Contextes géographiques très différents")

=== DIAGNOSTIC COMPLET ===

FICHIER 1: ghs_gaza_pop_added_full_extent.tif
CRS: EPSG:4326
Dimensions: 419 x 450 pixels
Transform: | 0.00, 0.00, 34.22|
| 0.00,-0.00, 31.59|
| 0.00, 0.00, 1.00|
Bounds: BoundingBox(left=34.21874988686396, bottom=31.219583410589177, right=34.56791655214766, top=31.594583409079952)

Coordonnées géographiques:
  Sud-Ouest: 31.219583°N, 34.218750°E
  Nord-Est:  31.594583°N, 34.567917°E

Dimensions réelles (méthode Haversine):
  Largeur: 33.20 km
  Hauteur: 41.70 km
  Surface: 1384.51 km²

Résolution des pixels:
  Largeur pixel: 0.00083333° = 79.2 m
  Hauteur pixel: 0.00083333° = 92.8 m

Analyse des données:
  Type: float32
  Min brut: 0.00
  Max brut: 632.09
  Pixels totaux: 188,550
  Pixels valides: 50,754 (26.9%)
  Min valide: 0.00
  Max valide: 632.09
  Moyenne valide: 41.24
  Surface couverte par les données: 372.78 km²

FICHIER 2: ghs_gaza_strip.tif
CRS: EPSG:4326
Dimensions: 419 x 450 pixels
Transform: | 0.00, 0.00, 34.22|
| 0.00,-0.00, 31.59|
| 0.00, 0.

In [13]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LinearSegmentedColormap
import os
from math import radians, cos, sin, asin, sqrt

# ================================
# PARAMÈTRES À AJUSTER
# ================================
downsample_factor = 2  # 1=max détail, 2=équilibré, 3=rapide
height_multiplier = 3.0  # Multiplicateur pour la hauteur des barres (1.0=normal, 3.0=triple hauteur)
create_small_multiple = True  # True pour créer l'aperçu 2x2 en plus des exports individuels
figure_height_ratio = 1.2  # Ajuste la hauteur de la figure (1.0=normal, 1.5=plus haut)
remove_z_limits = True  # True pour supprimer les limites Z et laisser les barres à leur vraie hauteur

# ================================
# CONFIGURATION
# ================================

# Tes fichiers
geotiff_files = [
    'data_input/data_processed/ghs_gaza_pop_added_full_extent.tif',
    'data_input/data_processed/ghs_gaza_strip.tif', 
    'data_input/data_processed/ghs_geneva.tif',
    'data_input/data_processed/ghs_schaffhausen.tif'
]

# Créer le dossier de sortie
os.makedirs('data_output/visualisation_3d', exist_ok=True)

# Create custom colormap with brighter colors
custom_colors_darkmode = [
    (0.0, '#575247'),     # Light beige (start at 0)
    (0.1, '#691A1F'),     # Medium green (middle)
    (1.0, '#E78D38')      # Dark green (end at 1)
]
custom_colors = [
    (0.0, '#DCD8C9'),     # Light beige (start at 0)
    (0.1, '#95D0CF'),     # Medium green (middle)
    (1.0, '#A757A3')      # Dark green (end at 1)
]

def deg_to_km(lat1, lon1, lat2, lon2):
    """Calcul précis de distance en km entre deux points (formule Haversine)"""
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Rayon de la Terre en km
    return c * r

def calculate_real_area_km2(bounds):
    """Calcule la superficie réelle en km² avec Haversine"""
    left, bottom, right, top = bounds
    width_km = deg_to_km(bottom, left, bottom, right)
    height_km = deg_to_km(bottom, left, top, left)
    return width_km * height_km

# First pass: find global min and max values and pixel sizes
print("=== ANALYSE GLOBALE ===")
all_data = []
pixel_sizes = []
areas_km2 = []
file_info = []

for file in geotiff_files:
    with rasterio.open(file) as src:
        data = src.read(1)
        # Garder les zéros, supprimer seulement les négatifs
        data = np.where(data < 0, np.nan, data)
        all_data.append(data)
        
        transform = src.transform
        pixel_size = (abs(transform[0]), abs(transform[4]))
        pixel_sizes.append(pixel_size)
        
        # Calcul correct de la superficie avec Haversine
        area = calculate_real_area_km2(src.bounds)
        areas_km2.append(area)
        
        # Informations sur les données valides
        valid_pixels = np.sum(~np.isnan(data))
        covered_area = (valid_pixels * pixel_size[0] * pixel_size[1] * 111.32**2)  # Approximation
        
        file_info.append({
            'name': file.split('/')[-1],
            'total_area': area,
            'valid_pixels': valid_pixels,
            'max_density': np.nanmax(data),
            'mean_density': np.nanmean(data[~np.isnan(data)]) if valid_pixels > 0 else 0
        })
        
        print(f"{file.split('/')[-1]:30} {area:8.1f} km² (max densité: {np.nanmax(data):6.1f})")

vmin = min(np.nanmin(data) for data in all_data)
vmax = max(np.nanmax(data) for data in all_data)

print(f"\nÉchelle globale des densités: {vmin:.1f} à {vmax:.1f}")
print(f"Multiplicateur de hauteur: {height_multiplier}x")
print(f"Limites Z supprimées: {remove_z_limits}")

# Find the largest dimensions to maintain scale
max_pixel_size = max(max(sizes) for sizes in pixel_sizes)
scale_factors = [(max_pixel_size/px[0], max_pixel_size/px[1]) for px in pixel_sizes]

print(f"\nFacteurs d'échelle (pour conserver les proportions géographiques):")
for i, (file, scale) in enumerate(zip(geotiff_files, scale_factors)):
    print(f"  {file.split('/')[-1]:30} {scale[0]:.3f} x {scale[1]:.3f}")

def create_3d_plot(ax, data, scale, area_km2, clean_name, file_info_item, mode='light', individual=True):
    """Fonction pour créer un plot 3D"""
    
    # Select appropriate colormap based on mode
    colors = custom_colors_darkmode if mode == 'dark' else custom_colors
    custom_cmap = LinearSegmentedColormap.from_list('custom', colors)
    
    if individual:
        # Make both figure and axis background transparent (pour exports individuels)
        ax.patch.set_alpha(0.0)
        # Set axis pane colors to transparent
        ax.xaxis.pane.set_facecolor((0.0, 0.0, 0.0, 0.0))
        ax.yaxis.pane.set_facecolor((0.0, 0.0, 0.0, 0.0))
        ax.zaxis.pane.set_facecolor((0.0, 0.0, 0.0, 0.0))
    
    # Downsample the data
    data_downsampled = data[::downsample_factor, ::downsample_factor]
    
    # Create coordinates for bars
    y, x = np.meshgrid(np.arange(data_downsampled.shape[1]), 
                      np.arange(data_downsampled.shape[0]))
    x = x.flatten() * downsample_factor * scale[0]
    y = y.flatten() * downsample_factor * scale[1]
    z = np.zeros_like(x)
    dx = dy = downsample_factor * max(scale) * 0.95
    dz = data_downsampled.flatten() * height_multiplier  # Appliquer le multiplicateur de hauteur
    
    # Remove only NaN values (keep zeros)
    mask = ~np.isnan(dz)
    x, y, z, dz = x[mask], y[mask], z[mask], dz[mask]
    
    print(f"    {len(dz):,} barres (dont {np.sum(dz == 0):,} zéros)")
    
    if len(dz) > 0:
        # Create the bar plot
        bars = ax.bar3d(x, y, z, dx, dy, dz, 
                        zsort='max',
                        shade=True)

        # Get the colors for all faces
        norm = plt.Normalize(vmin=vmin, vmax=vmax)  # Utiliser l'échelle originale pour les couleurs
        face_colors = custom_cmap(norm(dz / height_multiplier))  # Normaliser par le multiplicateur
        
        # Modify colors for each face (6 faces per bar)
        facecolors = np.zeros((len(dz) * 6, 4))
        for i in range(len(dz)):
            color = face_colors[i]
            dark_color = color * [0.6, 0.6, 0.6, 1.0]  # Face du dessous plus sombre
            facecolors[i*6:(i+1)*6] = color
            facecolors[i*6 + 2] = dark_color  # Face du dessous

        # Apply the modified colors
        bars.set_facecolors(facecolors)

        # Set boundaries with proper scaling
        x_min, x_max = np.min(x), np.max(x) + dx
        y_min, y_max = np.min(y), np.max(y) + dy
        x_padding = (x_max - x_min) * 0.02
        y_padding = (y_max - y_min) * 0.02
        
        # Set axis limits
        ax.set_xlim(x_min - x_padding, x_max + x_padding)
        ax.set_ylim(y_min - y_padding, y_max + y_padding)
        
        # Gestion intelligente des limites Z
        if remove_z_limits:
            # Laisser matplotlib déterminer automatiquement les limites Z
            pass
        else:
            # Limiter la hauteur pour une meilleure vue d'ensemble
            ax.set_zlim(0, vmax * height_multiplier * 0.8)
    
    ax.set_axis_off()
    ax.view_init(elev=30, azim=0)
    
    # Add title avec informations détaillées
    max_original = file_info_item['max_density']
    mean_original = file_info_item['mean_density']
    title = f"{clean_name}\n{area_km2:.0f} km² • max: {max_original:.0f} • moy: {mean_original:.1f}"
    fontsize = 14 if individual else 10
    ax.set_title(title, pad=20, fontsize=fontsize)
    
    return custom_cmap

# Create individual plots
print(f"\n=== CRÉATION DES EXPORTS INDIVIDUELS ===")
clean_names = []

for idx, (file, data, scale, area, info) in enumerate(zip(geotiff_files, all_data, scale_factors, areas_km2, file_info)):
    print(f"\nProcessing {file}")
    
    # Clean name
    clean_name = file.split('/')[-1].replace('ghs_', '').replace('.tif', '')
    if 'gaza_pop_added_full_extent' in clean_name:
        clean_name = 'Gaza (étendu)'
    elif 'gaza_strip' in clean_name:
        clean_name = 'Gaza (bande)'
    elif 'geneva' in clean_name:
        clean_name = 'Genève'
    elif 'schaffhausen' in clean_name:
        clean_name = 'Schaffhouse'
    
    clean_names.append(clean_name)
    
    # Create both light and dark mode versions
    for mode in ['light', 'dark']:
        print(f"  Creating {mode} mode version...")
        
        # Ajuster la hauteur de la figure selon le paramètre
        fig_height = int(10 * figure_height_ratio)
        fig = plt.figure(figsize=(12, fig_height), dpi=300)
        fig.patch.set_alpha(0.0)
        ax = fig.add_subplot(111, projection='3d')
        
        custom_cmap = create_3d_plot(ax, data, scale, area, clean_name, info, mode, individual=True)
        
        # Add colorbar
        cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
        sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
        cbar = fig.colorbar(sm, cax=cbar_ax)
        cbar.set_label('Densité de population\n(hab./km²)', fontsize=12)
        
        # Adjust layout
        plt.subplots_adjust(left=0.02, right=0.9)
        
        # Save with appropriate suffix
        suffix = '_dm' if mode == 'dark' else ''
        safe_name = clean_name.lower().replace(' ', '_').replace('(', '').replace(')', '')
        output_filename = f"data_output/visualisation_3d/{safe_name}_h{height_multiplier:.1f}x{suffix}.png"
        plt.savefig(output_filename, 
                    dpi=300, 
                    bbox_inches='tight',
                    pad_inches=0.1,
                    transparent=True,
                    facecolor='none')
        print(f"    Saved: {output_filename}")
        
        plt.close()

# Create small multiple overview
if create_small_multiple:
    print(f"\n=== CRÉATION DE L'APERÇU 2x2 ===")
    
    for mode in ['light', 'dark']:
        fig_height = int(12 * figure_height_ratio)
        fig = plt.figure(figsize=(16, fig_height), dpi=300)
        if mode == 'dark':
            fig.patch.set_facecolor('#2E2E2E')
        else:
            fig.patch.set_facecolor('white')
        
        title_text = f'Comparaison des densités de population'
        if height_multiplier != 1.0:
            title_text += f' (hauteur {height_multiplier}x)'
        fig.suptitle(title_text, fontsize=16, y=0.95)
        
        for idx, (data, scale, area, info, clean_name) in enumerate(zip(all_data, scale_factors, areas_km2, file_info, clean_names)):
            ax = fig.add_subplot(2, 2, idx + 1, projection='3d')
            custom_cmap = create_3d_plot(ax, data, scale, area, clean_name, info, mode, individual=False)
        
        # Add single colorbar
        cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
        sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
        cbar = fig.colorbar(sm, cax=cbar_ax)
        cbar.set_label('Densité de population (hab./km²)', fontsize=12)
        
        # Adjust layout
        plt.subplots_adjust(wspace=0.1, hspace=0.1, left=0.05, right=0.9, top=0.9, bottom=0.05)
        
        # Save overview
        suffix = '_dm' if mode == 'dark' else ''
        overview_filename = f"data_output/visualisation_3d/overview_2x2_h{height_multiplier:.1f}x{suffix}.png"
        plt.savefig(overview_filename, dpi=300, bbox_inches='tight')
        print(f"Aperçu 2x2 sauvé: {overview_filename}")
        
        plt.close()

print(f"\n=== TERMINÉ ===")
print(f"Paramètres utilisés:")
print(f"  - Facteur de sous-échantillonnage: {downsample_factor}")
print(f"  - Multiplicateur de hauteur: {height_multiplier}x")
print(f"  - Ratio hauteur figure: {figure_height_ratio}")
print(f"  - Limites Z supprimées: {remove_z_limits}")
print(f"  - Aperçu 2x2 créé: {create_small_multiple}")
print(f"\nSuperficies réelles calculées:")
for name, area in zip(clean_names, areas_km2):
    print(f"  {name}: {area:.0f} km²")
print(f"\nToutes les visualisations sauvées dans: data_output/visualisation_3d/")

=== ANALYSE GLOBALE ===
ghs_gaza_pop_added_full_extent.tif   1384.5 km² (max densité:  632.1)
ghs_gaza_strip.tif               1384.5 km² (max densité:  459.4)
ghs_geneva.tif                    718.3 km² (max densité:  289.9)
ghs_schaffhausen.tif             1006.9 km² (max densité:   93.3)

Échelle globale des densités: 0.0 à 632.1
Multiplicateur de hauteur: 3.0x
Limites Z supprimées: True

Facteurs d'échelle (pour conserver les proportions géographiques):
  ghs_gaza_pop_added_full_extent.tif 1.000 x 1.000
  ghs_gaza_strip.tif             1.000 x 1.000
  ghs_geneva.tif                 1.000 x 1.000
  ghs_schaffhausen.tif           1.000 x 1.000

=== CRÉATION DES EXPORTS INDIVIDUELS ===

Processing data_input/data_processed/ghs_gaza_pop_added_full_extent.tif
  Creating light mode version...
    12,696 barres (dont 10,468 zéros)
    Saved: data_output/visualisation_3d/gaza_étendu_h3.0x.png
  Creating dark mode version...
    12,696 barres (dont 10,468 zéros)
    Saved: data_output/visua

## version retravaillée avec openai

In [16]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import rasterio
from rasterio.transform import Affine
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# ================================
# PARAMÈTRES
# ================================
downsample_factor = 2       # 1=max détail
height_multiplier = 2.0     # facteur vertical (visuel) identique partout
create_small_multiple = True

geotiff_files = [
    'data_input/data_processed/ghs_gaza_pop_added_full_extent.tif',
    'data_input/data_processed/ghs_gaza_strip.tif',
    'data_input/data_processed/ghs_geneva.tif',
    'data_input/data_processed/ghs_schaffhausen.tif'
]

out_dir = 'data_output/visualisation_3d/openai_version'
os.makedirs(out_dir, exist_ok=True)

# Colormaps (clair/dark)
custom_colors_dark = [(0.0, '#575247'), (0.1, '#691A1F'), (1.0, '#E78D38')]
custom_colors_light = [(0.0, '#DCD8C9'), (0.1, '#95D0CF'), (1.0, '#A757A3')]

def name_cleaner(p):
    n = os.path.basename(p).replace('ghs_', '').replace('.tif', '')
    if 'gaza_pop_added_full_extent' in n: return 'Gaza (étendu)'
    if 'gaza_strip' in n: return 'Gaza (bande)'
    if 'geneva' in n: return 'Genève'
    if 'schaffhausen' in n: return 'Schaffhouse'
    return n

# ================================
# UTILS DISTANCES & AIRES
# ================================
def km_per_degree(lat_deg):
    """Retourne (km/°lon, km/°lat) à la latitude donnée (approx locale)."""
    km_lat = 111.32
    km_lon = 111.32 * np.cos(np.radians(lat_deg))
    return km_lon, km_lat

def pixel_vectors_from_affine(transform: Affine):
    """
    Vecteurs de pixel en unités du CRS :
    - v_col = déplacement quand on augmente la colonne de +1
    - v_row = déplacement quand on augmente la ligne de +1
    """
    # Affine: [a, b, c; d, e, f; 0, 0, 1]
    a, b, c, d, e, f = transform.a, transform.b, transform.c, transform.d, transform.e, transform.f
    v_col = np.array([a, d])  # déplacement en X,Y quand col += 1
    v_row = np.array([b, e])  # déplacement en X,Y quand row += 1
    origin = np.array([c, f]) # coord du pixel (0,0) (coin supérieur gauche)
    return v_col, v_row, origin

def affine_det_km2_per_pixel(transform: Affine, crs_is_metric: bool, lat_for_deg=None):
    """
    Aire d'un pixel en km² via le déterminant de l'affine.
    - En métrique: |det| (m²) → /1e6
    - En géo: convertit chaque pixel selon km/°; si lat_for_deg donné (scalaire ou array par ligne),
      on renvoie None et on utilisera la somme par lignes plus précise (voir compute_area_km2_geo).
    """
    det = transform.a * transform.e - transform.b * transform.d
    if crs_is_metric:
        return abs(det) / 1_000_000.0  # m² → km²
    return None  # en géo on fait mieux par lignes

def compute_area_km2_geo(transform: Affine, height, width):
    """
    Aire en km² pour CRS géographique (°) en sommant par ligne:
    aire_ligne = width * (km/°lon à la latitude de la ligne * |px_lon_deg|) * (km/°lat * |px_lat_deg|)
    en tenant compte de l'orientation de l'affine (b,d souvent 0 pour rasters north-up).
    """
    a, b, c, d, e, f = transform.a, transform.b, transform.c, transform.d, transform.e, transform.f
    # Taille en degrés des pas colonne et ligne (norme des vecteurs)
    # (si rotation nulle, |a| ~ px_lon_deg, |e| ~ px_lat_deg)
    px_lon_deg = np.hypot(a, d)
    px_lat_deg = np.hypot(b, e)

    # Latitude du centre de chaque ligne (on évalue km/°lon à cette latitude)
    # Coord du centre du pixel (row, col) : x = c + a*(col+0.5) + b*(row+0.5), y = f + d*(col+0.5) + e*(row+0.5)
    # Pour la latitude, on peut prendre la coord y au milieu de la ligne (col ~ (width-1)/2)
    mid_col = (width - 1) / 2.0
    row_idx = np.arange(height)
    lat_line = f + d*(mid_col + 0.5) + e*(row_idx + 0.5)

    km_lat = 111.32
    km_lon_per_line = 111.32 * np.cos(np.radians(lat_line))
    # Aire d'un pixel pour chaque ligne (km²)
    px_area_line_km2 = (px_lon_deg * km_lon_per_line) * (px_lat_deg * km_lat)
    # Total (somme sur lignes * nb de colonnes)
    return float(np.nansum(px_area_line_km2) * 1.0) * width

def coords_km_from_affine(transform: Affine, shape, crs_is_metric):
    """
    Calcule les coordonnées X,Y (centres de pixels) et les tailles dx,dy (empreinte) en KM,
    en respectant l'orientation d'origine (pas d'inversion arbitraire).
    """
    rows, cols = shape
    v_col, v_row, origin = pixel_vectors_from_affine(transform)

    # indices sous-échantillonnés
    ii = np.arange(0, rows, downsample_factor)
    jj = np.arange(0, cols, downsample_factor)
    JJ, II = np.meshgrid(jj, ii)  # JJ: cols, II: rows

    # coordonnées du centre des pixels (en unités CRS)
    centers = origin[None, None, :] + (JJ + 0.5)[..., None] * v_col[None, None, :] + (II + 0.5)[..., None] * v_row[None, None, :]
    Xu = centers[..., 0].reshape(-1)  # unités CRS
    Yu = centers[..., 1].reshape(-1)

    # tailles (vecteurs) pour un pixel (en unités CRS)
    # on prend 95% pour un léger espacement visuel
    vcol_len = np.hypot(v_col[0], v_col[1])
    vrow_len = np.hypot(v_row[0], v_row[1])

    if crs_is_metric:
        # unités en m → km
        Xkm = Xu / 1000.0
        Ykm = Yu / 1000.0
        dx = np.full_like(Xkm, vcol_len * downsample_factor / 1000.0 * 0.95)
        dy = np.full_like(Ykm, vrow_len * downsample_factor / 1000.0 * 0.95)
    else:
        # unités en degrés → km (conversion locale)
        # On convertit séparément lon/lat en km en évaluant km/° près de chaque point (approx)
        km_lon, km_lat = km_per_degree(Yu)  # Yu = latitude en degrés
        Xkm = Xu * km_lon
        Ykm = Yu * km_lat
        dx = np.full_like(Xkm, vcol_len * downsample_factor * (111.32 * np.cos(np.radians(np.mean(Yu)))) * 0.95)
        dy = np.full_like(Ykm, vrow_len * downsample_factor * 111.32 * 0.95)

    # Pour que toutes les scènes démarrent à 0,0 tout en gardant orientation relative,
    # on translate seulement (pas de flip): (minima peuvent être > ou < 0 selon orientation)
    Xkm = Xkm - np.nanmin(Xkm)
    Ykm = Ykm - np.nanmin(Ykm)

    return Xkm, Ykm, dx, dy

# ================================
# LECTURE + ANALYSE GLOBALE
# ================================
rasters = []
vals_min, vals_max = [], []

print("=== ANALYSE GLOBALE ===")
for fp in geotiff_files:
    with rasterio.open(fp) as src:
        arr = src.read(1).astype(float)
        arr[arr < 0] = np.nan  # garde 0, vire négatifs
        transform = src.transform
        crs = src.crs
        crs_is_metric = (crs is not None) and (not crs.is_geographic)

        rows, cols = arr.shape

        # Aire en km²
        px_det_km2 = affine_det_km2_per_pixel(transform, crs_is_metric)
        if px_det_km2 is not None:
            area_km2 = px_det_km2 * rows * cols
        else:
            area_km2 = compute_area_km2_geo(transform, rows, cols)

        rasters.append({
            'path': fp,
            'name': name_cleaner(fp),
            'data': arr,
            'transform': transform,
            'crs_is_metric': crs_is_metric,
            'area_km2': float(area_km2)
        })
        vals_min.append(np.nanmin(arr))
        vals_max.append(np.nanmax(arr))
        print(f"{os.path.basename(fp)}: {area_km2:.2f} km²; shape={rows}x{cols}; CRS_metric={crs_is_metric}")

# Échelle couleur globale (valeurs brutes)
vmin = float(np.nanmin(vals_min))
vmax = float(np.nanmax(vals_max))
print(f"Échelle couleur (valeurs brutes) : min={vmin:.3f}, max={vmax:.3f}")
print(f"Hauteur = valeur × {height_multiplier:g}")

# ================================
# PLOT 3D
# ================================
def create_3d_plot(ax, r, mode='light', individual=True):
    colors = custom_colors_dark if mode == 'dark' else custom_colors_light
    cmap = LinearSegmentedColormap.from_list('custom', colors)

    data = r['data']
    transform = r['transform']
    crs_is_metric = r['crs_is_metric']
    area_km2 = r['area_km2']
    name = r['name']

    if individual:
        ax.patch.set_alpha(0.0)
        for pane in (ax.xaxis.pane, ax.yaxis.pane, ax.zaxis.pane):
            pane.set_facecolor((0, 0, 0, 0))

    # Sous-échantillonnage des données
    data_ds = data[::downsample_factor, ::downsample_factor]

    # Coordonnées & tailles à partir de l'affine → en km, orientation conservée
    Xkm, Ykm, dx, dy = coords_km_from_affine(transform, data.shape, crs_is_metric)

    z = np.zeros_like(Xkm)
    dz = data_ds.flatten() * height_multiplier  # extrusion visuelle

    mask = ~np.isnan(dz)
    Xkm, Ykm, z, dx, dy, dz = Xkm[mask], Ykm[mask], z[mask], dx[mask], dy[mask], dz[mask]

    if dz.size > 0:
        bars = ax.bar3d(Xkm, Ykm, z, dx, dy, dz, zsort='max', shade=True)
        # Couleur = valeur brute (pas *height_multiplier)
        values_raw = dz / height_multiplier
        norm = plt.Normalize(vmin=vmin, vmax=vmax)
        face_cols = cmap(norm(values_raw))

        # Légère ombre sur une face
        facecolors = np.zeros((len(dz) * 6, 4))
        for i, c in enumerate(face_cols):
            darker = c * [0.6, 0.6, 0.6, 1.0]
            facecolors[i*6:(i+1)*6] = c
            facecolors[i*6 + 2] = darker
        bars.set_facecolors(facecolors)

        ax.set_zlim(0, vmax * height_multiplier * 0.9)

        # Limites XY (padding léger)
        x_min, x_max = np.nanmin(Xkm), np.nanmax(Xkm) + (dx[0] if dx.size else 0)
        y_min, y_max = np.nanmin(Ykm), np.nanmax(Ykm) + (dy[0] if dy.size else 0)
        ax.set_xlim(x_min - 0.02*(x_max-x_min), x_max + 0.02*(x_max-x_min))
        ax.set_ylim(y_min - 0.02*(y_max-y_min), y_max + 0.02*(y_max-y_min))

    ax.set_axis_off()
    ax.view_init(elev=30, azim=0)
    ax.set_title(f"{name}\nSurface: {area_km2:.1f} km²", pad=14, fontsize=(14 if individual else 10))
    return cmap

# ================================
# EXPORTS INDIVIDUELS
# ================================
print("\n=== EXPORTS INDIVIDUELS ===")
for r in rasters:
    for mode in ['light', 'dark']:
        fig = plt.figure(figsize=(12, 10), dpi=300)
        fig.patch.set_alpha(0.0)
        ax = fig.add_subplot(111, projection='3d')
        cmap = create_3d_plot(ax, r, mode=mode, individual=True)

        cax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
        cb = fig.colorbar(sm, cax=cax)
        cb.set_label('Densité de population (hab./km²)', fontsize=12)

        plt.subplots_adjust(left=0.02, right=0.9)
        suffix = '_dm' if mode == 'dark' else ''
        safe = r['name'].lower().replace(' ', '_').replace('(', '').replace(')', '')
        out = os.path.join(out_dir, f"{safe}{suffix}.png")
        plt.savefig(out, dpi=300, bbox_inches='tight', pad_inches=0.1, transparent=True, facecolor='none')
        plt.close()
        print(f"  → {out}")

# ================================
# SMALL MULTIPLE 2×2
# ================================
if create_small_multiple:
    print("\n=== SMALL MULTIPLE 2×2 ===")
    for mode in ['light', 'dark']:
        fig = plt.figure(figsize=(16, 12), dpi=300)
        fig.patch.set_facecolor('#2E2E2E' if mode == 'dark' else 'white')
        fig.suptitle(f'Comparaison des densités (hauteur × {height_multiplier:g})',
                     fontsize=16, y=0.95, color=('white' if mode=='dark' else 'black'))

        cmap_last = None
        for i, r in enumerate(rasters, start=1):
            ax = fig.add_subplot(2, 2, i, projection='3d')
            cmap_last = create_3d_plot(ax, r, mode=mode, individual=False)

        cax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
        sm = plt.cm.ScalarMappable(cmap=cmap_last, norm=plt.Normalize(vmin=vmin, vmax=vmax))
        cb = fig.colorbar(sm, cax=cax)
        cb.set_label('Densité de population (hab./km²)', fontsize=12,
                     color=('white' if mode=='dark' else 'black'))
        if mode == 'dark':
            cb.ax.yaxis.set_tick_params(colors='white')

        plt.subplots_adjust(wspace=0.1, hspace=0.1, left=0.05, right=0.9, top=0.9, bottom=0.05)
        out = os.path.join(out_dir, f"overview_2x2{'_dm' if mode=='dark' else ''}.png")
        plt.savefig(out, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"  → {out}")

print("\n=== TERMINÉ ===")
print(f"downsample_factor={downsample_factor} | height_multiplier={height_multiplier} | small_multiple={create_small_multiple}")
print(f"Exports: {out_dir}")

=== ANALYSE GLOBALE ===
ghs_gaza_pop_added_full_extent.tif: 1384.86 km²; shape=450x419; CRS_metric=False
ghs_gaza_strip.tif: 1384.86 km²; shape=450x419; CRS_metric=False
ghs_geneva.tif: 718.32 km²; shape=284x425; CRS_metric=False
ghs_schaffhausen.tif: 1006.75 km²; shape=307x566; CRS_metric=False
Échelle couleur (valeurs brutes) : min=0.000, max=632.088
Hauteur = valeur × 2

=== EXPORTS INDIVIDUELS ===
  → data_output/visualisation_3d/openai_version/gaza_étendu.png
  → data_output/visualisation_3d/openai_version/gaza_étendu_dm.png
  → data_output/visualisation_3d/openai_version/gaza_bande.png
  → data_output/visualisation_3d/openai_version/gaza_bande_dm.png
  → data_output/visualisation_3d/openai_version/genève.png
  → data_output/visualisation_3d/openai_version/genève_dm.png
  → data_output/visualisation_3d/openai_version/schaffhouse.png
  → data_output/visualisation_3d/openai_version/schaffhouse_dm.png

=== SMALL MULTIPLE 2×2 ===
  → data_output/visualisation_3d/openai_version/overvie